In [1]:
#!/usr/bin/env python

import os
import argparse
from lstchain.io.io import read_dl2_to_pyirf, dl2_params_lstcam_key
import logging
import operator
from lstchain.mc import plot_utils
import ctaplot
from ctaplot.plots import plot_angular_resolution_per_energy

import numpy as np
from astropy import table
import astropy.units as u
from astropy.io import fits
import matplotlib as mpl
import matplotlib.pyplot as plt
from astropy.table import QTable
import pandas as pd

from pyirf.io.eventdisplay import read_eventdisplay_fits
from pyirf.binning import (
    create_bins_per_decade,
    add_overflow_bins,
    create_histogram_table,
)
from pyirf.cuts import calculate_percentile_cut, evaluate_binned_cut
from pyirf.sensitivity import calculate_sensitivity
from pyirf.utils import calculate_theta, calculate_source_fov_offset
from pyirf.benchmarks import energy_bias_resolution, angular_resolution

from pyirf.cuts import calculate_percentile_cut, evaluate_binned_cut
from pyirf.sensitivity import calculate_sensitivity, estimate_background
from pyirf.utils import calculate_theta, calculate_source_fov_offset
from pyirf.benchmarks import energy_bias_resolution, angular_resolution

from pyirf.spectral import (
    calculate_event_weights,
    PowerLaw,
    CRAB_HEGRA,
    IRFDOC_PROTON_SPECTRUM
)
from pyirf.cut_optimization import optimize_gh_cut

from pyirf.irf import (
    effective_area_per_energy,
    energy_dispersion,
    psf_table,
    background_2d,
)

from pyirf.io import (
    create_aeff2d_hdu,
    create_psf_table_hdu,
    create_energy_dispersion_hdu,
    create_rad_max_hdu,
    create_background_2d_hdu,
)


/Users/davidsudm/opt/anaconda3/envs/cta-lstchain_sipm.v.0.6.3_pyirf.v.0.4/lib/python3.7/site-packages/ctapipe/tools/stage1.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
# Aleksic et al., 2015, JHEAP, (https://doi.org/10.1016/j.jheap.2015.01.002), fit od 0.1-20 TeV
def crab_magic2015_dNdE(energy):
    spectrum = 3.23 * 10**-11 * energy**(-2.47 - 0.24 * np.log10(energy))   # TeV-1 cm-2 s-1
    #spectrum = spectrum / 1.602176487   # erg-1 cm-2 s-1
    # spectrum = spectrum * 10000 # TeV-1 m-2 s-1
    return spectrum


def plot_magic_sensitivity(ax=None, **kwargs):

    if ax is None:
        fig, ax = plt.subplots()

    magic_table = ctaplot.get_magic_sensitivity()
    magic_table['e_err_lo'] = magic_table['e_center'] - magic_table['e_min']
    magic_table['e_err_hi'] = magic_table['e_max'] - magic_table['e_center']

    key = 'lima_5off'

    if 'ls' not in kwargs and 'linestyle' not in kwargs:
        kwargs['ls'] = ''
    kwargs.setdefault('label', f'MAGIC (Aleksić et al, 2016)')

    k = 'sensitivity_' + key
    ax.errorbar(
        magic_table['e_center'].to_value(u.TeV),
        y=(magic_table['e_center'] ** 2 * magic_table[k]).to_value(u.Unit('TeV cm-2 s-1')),
        xerr=[magic_table['e_err_lo'].to_value(u.TeV), magic_table['e_err_hi'].to_value(u.TeV)],
        yerr=(magic_table['e_center'] ** 2 * magic_table[f'{k}_err']).to_value(u.Unit('TeV cm-2 s-1')),
        **kwargs)

    e_smooth = np.logspace(np.log10(0.001), np.log10(100), 100)
    ax.plot(e_smooth, e_smooth ** 2 * crab_magic2015_dNdE(e_smooth), '-', alpha=0.5, color='grey',
            label='100% Crab')  # 100 %
    ax.plot(e_smooth, e_smooth ** 2 * 0.1 * crab_magic2015_dNdE(e_smooth), '--', alpha=0.5, color='grey',
            label='10% Crab')  # 10 %
    ax.plot(e_smooth, e_smooth ** 2 * 0.01 * crab_magic2015_dNdE(e_smooth), '-.', alpha=0.5, color='grey',
            label='1% Crab')  # 1 %

    ax.set_xlabel(r"$E_{R}$ [TeV]")
    ax.set_ylabel(r"$E^2$ Flux Sensitivity [TeV $cm^{-2} s^{-1}$]")
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlim(1e-2, 1e2)
    ax.set_ylim(1e-13, 1e-9)
    ax.grid('both')

    return ax


def plot_sensitivity(sensitivity_tables, labels, colors, markers, output_dir):
    """
    Makes sensitivity plot of a list of sensitivity tables
    :param sensitivity_tables:      a list of sensitivity tables
    :param labels:                  labels of the list of data
    :param colors:                  color of the list of data
    :param markers:                 markers of the list of data
    :param output_dir:              output directory where to store the plot
    :return:
        nothing, it just save the plot
    """

    fig, ax = plt.subplots()
    plot_magic_sensitivity(ax=ax, color='tab:gray')
    for i, label in enumerate(labels):
        xx = sensitivity_tables[i]['reco_energy_center'].to_value()
        yy = xx ** 2 * sensitivity_tables[i]['flux_sensitivity'].to_value()
        xx_err = 0.5 * (sensitivity_tables[i]['reco_energy_high'] - sensitivity_tables[i]['reco_energy_low']).to_value()
        #yy_err = xx ** 2 * [sensitivity_tables[i]['flux_sensitivity_lower_err'].to_value(),
        #                    sensitivity_tables[i]['flux_sensitivity_upper_err'].to_value()]
        #ax.errorbar(x=xx, y=yy, xerr=xx_err, yerr=yy_err, label=labels[i], ecolor=colors[i], color=colors[i], fmt=markers[i])
        ax.errorbar(x=xx, y=yy, xerr=xx_err, label=labels[i], ecolor=colors[i], color=colors[i], fmt=markers[i])

    ax.legend(ncol=1, loc=1)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'flux_sensitivity.png'), dpi=800)
    plt.close(fig)


def plot_angular_resolution(gamma_tables, proton_tables, labels, colors, markers, output_dir):
    """
    Makes the angular resolution plot of list of tables
    :param gamma_tables:
    :param proton_tables:
    :param labels:                  labels of the list of data
    :param colors:                  color of the list of data
    :param markers:                 markers of the list of data
    :param output_dir:              output directory where to store the plot
    :return:
        nothing, it just save the plot
    """

    fig, ax = plt.subplots()

    for i, gamma_table in enumerate(gamma_tables):

        # Events that passed out the gammaness cut (mix of protons and gammas)
        events = table.vstack(gamma_tables[i][gamma_tables[i]["selected_gh"]],
                              proton_tables[i][proton_tables[i]["selected_gh"]])

        plot_angular_resolution_per_energy(reco_alt=events['reco_alt'].to_value(),
                                           reco_az=events['reco_az'].to_value(),
                                           true_alt=events['true_alt'].to_value(),
                                           true_az=events['true_az'].to_value(),
                                           reco_energy=events['reco_energy'].to_value(),
                                           percentile=68.27,
                                           confidence_level=0.95,
                                           bias_correction=False,
                                           ax=ax,
                                           color=colors[i],
                                           label=labels[i],
                                           marker=markers[i])

    ax.set_xlim(1e-2, 1e2)
    ax.grid('on', which="both")
    ax.set_title(None)
    ax.legend()

    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'angular_resolution.png'), dpi=800)
    plt.close(fig)


def plot_theta2_distribution(gamma_tables, proton_tables, labels, colors, output_dir):
    """
    Makes the angular resolution plot of list of tables
    :param gamma_tables:
    :param proton_tables:
    :param labels:                  labels of the list of data
    :param colors:                  color of the list of data
    :param markers:                 markers of the list of data
    :param output_dir:              output directory where to store the plot
    :return:
        nothing, it just save the plot
    """

    fig, ax = plt.subplots()

    for i, gamma_table in enumerate(gamma_tables):

        # Events that passed out the gammaness cut (mix of protons and gammas)
        events = table.vstack(gamma_tables[i][gamma_tables[i]["selected_gh"]],
                              proton_tables[i][proton_tables[i]["selected_gh"]])

        ctaplot.plot_theta2(reco_alt=events['reco_alt'].to_value(),
                            reco_az=events['reco_az'].to_value(),
                            true_alt=events['true_alt'].to_value(),
                            true_az=events['true_az'].to_value(),
                            bias_correction=False,
                            ax=ax,
                            color=colors[i],
                            label=labels[i],
                            histtype='step')

    ax.set_title('')
    ax.grid('on', which="both")
    ax.legend()
    ax.set_yscale('log')

    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'theta2.png'), dpi=800)
    plt.close(fig)


def plot_energy_resolution(gamma_tables, labels, colors, markers, output_dir):
    """
    Makes the energy resolution plot of list of tables
    :param gamma_tables:
    :param labels:                  labels of the list of data
    :param colors:                  color of the list of data
    :param markers:                 markers of the list of data
    :param output_dir:              output directory where to store the plot
    :return:
        nothing, it just save the plot
    """

    fig, ax = plt.subplots()

    for i, gamma_table in enumerate(gamma_tables):

        ctaplot.plot_energy_resolution(true_energy=gamma_table['true_energy'].to_value(),
                                       reco_energy=gamma_table['reco_energy'].to_value(),
                                       ax=ax,
                                       bias_correction=False,
                                       color=colors[i],
                                       marker=markers[i],
                                       label=labels[i])

    ax.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'energy_resolution.png'), dpi=800)
    plt.close(fig)


def plot_energy_bias(gamma_tables, labels, colors, markers, output_dir):
    """
    Makes the energy bias plot of list of tables
    :param gamma_tables:
    :param labels:                  labels of the list of data
    :param colors:                  color of the list of data
    :param markers:                 markers of the list of data
    :param output_dir:              output directory where to store the plot
    :return:
        nothing, it just save the plot
    """

    fig, ax = plt.subplots()

    for i, gamma_table in enumerate(gamma_tables):

        ctaplot.plot_energy_bias(true_energy=gamma_table['true_energy'].to_value(),
                                 reco_energy=gamma_table['reco_energy'].to_value(),
                                 ax=ax,
                                 color=colors[i],
                                 marker=markers[i],
                                 label=labels[i])

    ax.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'energy_bias.png'), dpi=800)
    plt.close(fig)


In [3]:
# Data is the downloaded dl2 files from lstchain 
input_files = ['/Users/davidsudm/Desktop/dl2_files/david/data/sensitivity.hdf5',
               '/Users/davidsudm/Desktop/dl2_files/gabriel/data/sensitivity.hdf5',
               '/Users/davidsudm/Desktop/dl2_files/official/data/sensitivity.hdf5']

labels = ['david : int/leak 100/0.2',
          'gabriel : int/leak 100/0.2',
          'mabel : int/leak 100/0.2']

markers = ['o',
           's',
           '^']

colors = ['tab:blue', 'tab:red', 'tab:green']
output_dir = "/Users/davidsudm/Desktop/dl2_files/all/"

try:
    os.makedirs(output_dir, exist_ok=True)
    print("Directory ", output_dir, " Created ")
except FileExistsError:
    print("Directory ", output_dir, " already exists")

sensitivity = []

for i, file in enumerate(input_files):
    if i == 1:
        my_path = 'sensitivity_table'
    else:
        my_path = 'sensitivity/standard'
    sensitivity.append(QTable.read(file, path=my_path))
    
    #gamma_histo.append(QTable.read(file, path='histogram/gamma_on'))
    #proton_histo.append(QTable.read(file, path='histogram/proton'))
    #df_gamma.append(QTable.read(file, path='events/gamma_on'))
    #df_proton.append(QTable.read(file, path='events/proton'))


Directory  /Users/davidsudm/Desktop/dl2_files/all/  Created 


In [4]:
# Plot Flux Sensitivity
plot_sensitivity(sensitivity_tables=sensitivity,
                 labels=labels,
                 colors=colors,
                 markers=markers,
                 output_dir=output_dir)